In [ ]:
!pip install openai

In [5]:
import requests
import json
import os

from openai import OpenAI

In [11]:
client = OpenAI()

In [ ]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "developer",
            "content": [{
                "type": "text",
                "text": """
                    Users are playing an RPG. They will ask questions pertaining to the rules, setting, lore,
                    and current situation. You will accept the query and return a list of relevant keywords that
                    can be used to locate relevant sections of the source material.

                    You will NOT attempt to answer the questions yourself. Your only role is to extract useful keywords
                    to be used in a search. Organize them in order of likely relevance, from most relevant to least.

                    Return the keywords in a comma delimited format with no quote characters or extraneous punctuation.
                """
            }],
        },
        {
            "role": "user",
            "content": [{
                "type": "text",
                "text": "My Strength is 16. I am wearing full plate armor. Does this impact my Swim speed?"
            }]
        }
    ]
)

print(completion.choices[0].message)

In [ ]:
completion.choices[0].message.content

In [45]:
def elastic_request(data=None, method=None, url=None):
    if method == None:
        method = requests.get
    if data:
        data = json.dumps(data)    

    return method(f"https://192.168.1.153:9200/{url}",
                  headers={
                    "Content-Type": "application/json",
                    "Accept": "application/json",
                    "Authorization": f"ApiKey {os.getenv('ELASTIC_API_KEY')}"
                  },
                  verify=False,
                  data=data)

In [ ]:
response = elastic_request(url="players-handbook-embedded/_search",
                           data={
                                "query": {
                                    "match": {
                                        "attachment.content": {
                                            "query": completion.choices[0].message.content,
                                            "operator": "or",
                                            "boost": 0.7
                                        }
                                    }
                                },
                                "knn": {
                                    "field": "content-embedding",
                                    "k": 10,
                                    "boost": 0.3,
                                    "num_candidates": 10,
                                    "query_vector_builder": {
                                        "text_embedding": { 
                                            "model_id": "open-ai-embeddings", 
                                            "model_text": "My Strength is 16. I am wearing full plate armor. Does this impact my Swim speed?" 
                                        }
                                    }
                                }
                           })

In [47]:
hits = response.json()["hits"]["hits"]

In [ ]:
[h["_source"]["attachment"]["content"] for h in hits]

In [49]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "developer",
            "content": [{
                "type": "text",
                "text": """
                    Users are playing an RPG. They will ask questions pertaining to the rules, setting, lore,
                    and current situation. You will answer the question based on the content provided.
                    
                    Use only the content provided. Do not base your answers on anything other than the content.
                    Quote the content verbatim if possible (but only if it is relevant to the question!).
                    Otherwise, keep your answer as brief as possible. 
                """
            }],
        },
        {
            "role": "assistant",
            "content": [{ "type": "text", "text": h["_source"]["attachment"]["content"]} for h in hits[0:2]]
        },
        {
            "role": "user",
            "content": [{
                "type": "text",
                "text": "My Strength is 16. I am wearing full plate armor. Does this impact my Swim speed?"
            }]
        }
    ]
)

In [ ]:
completion.choices[0].message.content

In [51]:
benchmark_questions = [
    "Can I cast Hunter's Mark on an Invisible enemy?",
    "My Strength is 18. I will take a running jump. How far can I make it?",
    "My Strength is 18. I am wearing full plate armor. How does this impact my ability to make a running jump?",
    "My Strength is 16. I am wearing full plate armor. Does this impact my Swim speed?"
]